In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [100]:
import modeling_helper_class as mh
import modeling_helpers as mh2
import pandas as pd 
import data_preprocessing_helpers as dp

from sklearn.ensemble import RandomForestRegressor as rf

In [101]:
df = dp.get_preprocessed_data()

df['LivLotRatio'] = df['GrLivArea']/df['LotArea']
df['Spaciousness'] = (df['1stFlrSF'] + df['2ndFlrSF'])/df['TotRmsAbvGrd']
df['TotalOutsideSF'] = df['WoodDeckSF'] + df['OpenPorchSF'] + df['EnclosedPorch'] + df['3SsnPorch'] + df['ScreenPorch']
df['TotalArea'] = df.TotalBsmtSF+ df.TotalOutsideSF + df['1stFlrSF'] + df['2ndFlrSF']
df[['OverallQual', 'OverallCond', 'NeighborhoodScore', 'ExterScore']] = df[['OverallQual', 'OverallCond', 'NeighborhoodScore', 'ExterScore']] .astype('category')

porches = ['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']
df['Porches'] = df[porches].gt(0).sum(axis = 1)

a = pd.get_dummies(df['BldgType'])
b = a.mul(df['GrLivArea'], axis =0)
df = df.join(b)


df = df.drop(['QualScore', 'QualScore2', 'BsmtFinSF1', 'BsmtFinSF2'], axis = 1)


Loading ./../data/house_prices/train.csv
Data has (1460, 80) shape
Feature Engineering
Making New Features
Setting Column Type
Making new features using Training Set


In [102]:
import category_encoders as ce
cat_cols = [x for x in df.columns if df.dtypes[x] in ['category', 'object']]
print(len(cat_cols))
cat_cols = cat_cols + ['OverallQual', 'OverallCond', 'NeighborhoodScore', 'ExterScore']
print(len(cat_cols))

x, y = dp.split_data(df, 'SalePrice')
encoder = ce.LeaveOneOutEncoder(cols = cat_cols)
encoder.fit(x,y)
x2 = encoder.transform(x)

mo = mh.model_object(model=rf(), df = pd.concat([x2, y], axis = 1), 
                     target = 'SalePrice', scorer = mh2.scorer)


50
54
input data has 1460 rows
Welcome to model object


In [103]:
mo.get_best_model()

Validating Model RandomForestRegressor() with 100 features
Baseline Score: -0.140506936746074, 0.01277635234957068
Tuning
Tuning RandomForestRegressor(), with  100 features
Tuned Baseline Score: -0.13941323696203442, 0.012713452171107805
FeatSelecting
Performing RFE CV
Validating Model RandomForestRegressor() with 70 features
FS - BL Score : -0.14053704803965306, 0.013367246503308226
Tuning
Tuning RandomForestRegressor(), with  70 features
FS - BL - Tuned Score: -0.1393676566506268, 0.012821938962231058
Validating Model RandomForestRegressor(max_depth=74, max_features=538, n_estimators=817) with 70 features
Scoring Again: -0.13946958485056307, 0.012666513400147036
Selecting Features for Tuned Model
Performing RFE CV
Validating Model RandomForestRegressor(max_depth=45, max_features=135, n_estimators=626) with 88 features
FS - Tuned - Tuned Score : -0.13921279894588698, 0.012536338407059073
Tuning
Tuning RandomForestRegressor(max_depth=45, max_features=135, n_estimators=626), with  88 fe

In [1]:
def get_model_importance(model):
    importance_df = pd.DataFrame(
        model.feature_importances_, 
        index = model.feature_names_in_, 
        columns = ['importance_rf']) \
        .sort_values('importance_rf', ascending=False)

    return importance_df

In [2]:
a = get_model_importance(mo.model_bl)
b = get_model_importance(mo.model_bl_tuned)
c = get_model_importance(mo.model_blfs_tuned)
d = get_model_importance(mo.model_tunedfs_tuned)


c = a.merge(b, left_index=True, right_index=True, suffixes = ['_bl', '_bl_tuned'])
c['average_importance'] = c.sum(axis = 1)/2
c.sort_values('average_importance', ascending=False)
c.tail(20)
#c = pd.concat([a,b], axis=1)
#c['average_importance'] = 

NameError: name 'mo' is not defined

In [99]:
mo.tunedfs_tuning_res

AttributeError: 'model_object' object has no attribute 'tunedfs_tuning_res'

In [67]:
df['GrLivArea']

Id
1       1710
2       1262
3       1786
4       1717
5       2198
        ... 
1456    1647
1457    2073
1458    2340
1459    1078
1460    1256
Name: GrLivArea, Length: 1460, dtype: int64